In [49]:
import pandas
import numpy as np

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, IntSlider
import ipywidgets as widgets

import matplotlib.pyplot as plt
import scipy.stats as stats

def make_table(log):
    shuffles = sorted(log.shuffles.unique())
    data = [log.loc[log['shuffles'] == shuffle].value.to_numpy() for shuffle in shuffles]
    
    return np.array(shuffles), np.array(data)

def prepare_log(path):
    log = pandas.read_csv(path, header=None)
    log.head()

    log = log[log[1].str.contains('infty')]
    log[["type", "value"]] = log[1].str.split(expand=True)
    log[["type", "shuffles"]] = log["type"].str.split('on', expand=True)
    log["value"] = pandas.to_numeric(log["value"])
    log["shuffles"] = pandas.to_numeric(log["shuffles"])
    log = log[["type", "shuffles", "value"]]

    simple_infty = log.loc[log['type'] == 'infty']
    ultimate_infty = log.loc[log['type'] == 'ultimateinfty']
    
    return make_table(simple_infty), make_table(ultimate_infty)

def time_dashboard(paths, ultimate=False):
    logs = [prepare_log(path)[1 if ultimate else 0] for path in paths]
    
    miny, maxy = None, None
    for (x, y) in logs:
        miny = np.min(y) if miny is None else min(miny, np.min(y))
        maxy = np.max(y) if maxy is None else max(maxy, np.max(y))
    
    def plotter(time_point):
        plt.title('Time: {0}'.format(time_point))
        
        for (x, y) in logs:
            if y.shape[1] > time_point:
                plt.plot(x, y[:,time_point])
            else:
                plt.plot([], [])
        
        plt.xlabel('shuffles', fontsize=15)
        plt.ylabel('infty', fontsize=15)
        plt.ylim((miny - 1, maxy + 1))
        plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
        plt.rcParams['figure.figsize'] = [16, 8]
        plt.legend(paths)
        plt.show()

    lengths = [log[1].shape[1]-1 for log in logs]
    interact(plotter, time_point=widgets.IntSlider(min=0, max=max(lengths), step=1, value=0,
                                                   layout=Layout(width='100%')))

def shuffles_dashboard(paths, ultimate=False):
    logs = [prepare_log(path)[1 if ultimate else 0] for path in paths]
    
    shuffles_list = []
    for (x, y) in logs:
        shuffles_list.extend(x)
    shuffles_list = sorted(list(set(shuffles_list)))
    
    miny, maxy = None, None
    for (x, y) in logs:
        miny = np.min(y) if miny is None else min(miny, np.min(y))
        maxy = np.max(y) if maxy is None else max(maxy, np.max(y))
    
    def plotter(shuffles_idx):
        shuffles = shuffles_list[shuffles_idx]
        plt.title('Shuffles: {0}'.format(shuffles))
        
        for (x, y) in logs:
            if shuffles in x:
                idx = np.where(x == shuffles)[0][0]
                plt.plot(np.arange(y.shape[1]), y[idx,:])
            else:
                plt.plot([], [])
        
        plt.xlabel('time', fontsize=15)
        plt.ylabel('infty', fontsize=15)
        plt.ylim((miny - 1, maxy + 1))
        plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
        plt.rcParams['figure.figsize'] = [16, 8]
        plt.legend(paths)
        plt.show()

    interact(plotter, shuffles_idx=widgets.IntSlider(min=0, max=len(shuffles_list)-1, step=1, value=0,
                                                 layout=Layout(width='100%')))

In [55]:
paths = ['log_279.csv', 'log_280.csv', 'log_281.csv', 'log_282.csv',
         'log_283.csv', 'log_284.csv', 'log_285.csv']

# Wykresy

Dla ustalonego punktu w czasie.

In [56]:
time_dashboard(paths)

interactive(children=(IntSlider(value=0, description='time_point', layout=Layout(width='100%'), max=172), Outp…

In [52]:
time_dashboard(paths, ultimate=True)

interactive(children=(IntSlider(value=0, description='time_point', layout=Layout(width='100%'), max=172), Outp…

Dla ustalonej liczby tasowań.

In [57]:
shuffles_dashboard(paths)

interactive(children=(IntSlider(value=0, description='shuffles_idx', layout=Layout(width='100%'), max=7), Outp…

In [54]:
shuffles_dashboard(paths, ultimate=True)

interactive(children=(IntSlider(value=0, description='shuffles_idx', layout=Layout(width='100%'), max=1), Outp…